## 1数据初步分析

In [1]:
import pandas as pd

data = pd.read_csv('tmall_order_report.csv')
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


原始数据中有金额、地址和时间三种类型的数据，因此首先我们可以统计每个省份的订单量，观察哪些省份的订单量比较大，从而可以和快递公司谈合作降低成本。其次对订单的订单创建时间进行两种分析，第一种是随天数的变化趋势，这可以帮助预测未来的销量，第二种是二十四小时内的订单随时间变化趋势，这可以知晓支付高峰，为维护网站提供支持。

## 2数据细节和清洗

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


除去订单付款时间之外，其他的数据都没有空值，这是因为有一些订单在创建之后并没有付款。

In [3]:
data.columns = data.columns.str.strip()
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

In [4]:
data[data.duplicated()].count()
# 假设一行重复的数据中有None值，在单统计含有None值的这一列的时候，不会将这个None值算进去

订单编号        0
总金额         0
买家实际支付金额    0
收货地址        0
订单创建时间      0
订单付款时间      0
退款金额        0
dtype: int64

In [5]:
data.isnull().sum()

订单编号           0
总金额            0
买家实际支付金额       0
收货地址           0
订单创建时间         0
订单付款时间      3923
退款金额           0
dtype: int64

In [6]:
result = {}
result['总订单数'] = data['订单编号'].count()
result['已完成订单数'] = data['订单编号'][data['订单付款时间'].notnull()].count()
# 付款完成的订单
result['未付款订单数'] = data['订单编号'][data['订单付款时间'].isnull()].count()
# 未完成付款的订单
result['退款订单数'] = data['订单编号'][data['退款金额']>0].count()
# 付款完成订单里面有退款的
result['总订单金额'] = data['总金额'][data['订单付款时间'].notnull()].sum()
# 付款完成订单里面商品的原价和
result['总退款金额'] = data['退款金额'][data['订单付款时间'].notnull()].sum()
# 付款完成订单里退款数的和
result['总实际收入金额'] = data['买家实际支付金额'][data['订单付款时间'].notnull()].sum()
# 付款完成订单里买家实际支付的金额
result



{'总订单数': 28010,
 '已完成订单数': 24087,
 '未付款订单数': 3923,
 '退款订单数': 5646,
 '总订单金额': 2474823.0700000003,
 '总退款金额': 572335.9199999999,
 '总实际收入金额': 1902487.1500000001}

In [7]:
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

table = Table()

headers = ['总订单数', '总订单金额', '已完成订单数', '总实际收入金额', '退款订单数', '总退款金额', '成交率', '退货率']
rows = [
    [
        result['总订单数'], f"{result['总订单金额']/10000:.2f} 万", result['已完成订单数'], f"{result['总实际收入金额']/10000:.2f} 万",
        result['退款订单数'], f"{result['总退款金额']/10000:.2f} 万", 
        # 成交率是付款完成订单（包括退款）在总订单里面的占比，退货率是退货订单在已完成订单里的占比
        f"{result['已完成订单数']/result['总订单数']:.2%}",
        f"{result['退款订单数']/result['已完成订单数']:.2%}",
    ]
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况')
)
table.render_notebook()

总订单数,总订单金额,已完成订单数,总实际收入金额,退款订单数,总退款金额,成交率,退货率
28010,247.48 万,24087,190.25 万,5646,57.23 万,85.99%,23.44%


## 3数据可视化

In [15]:
data['收货地址'] = data['收货地址'].str.replace('自治区|维吾尔|回族|壮族|省', '')
# 原始数据里面会有'自治区|维吾尔|回族|壮族|省'等字样，这些是不被pyecharts识别的
result2 = data[data['订单付款时间'].notnull()].groupby('收货地址').agg({'订单编号':'count'})
result21 = result2.to_dict()['订单编号']
c = (
    Map()
    .add("订单量", [*result21.items()], "china", is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='地区分布'),
        visualmap_opts=opts.VisualMapOpts(max_=1000),            
    )
)
c.render_notebook()

我们可以看到，订单收货地主要集中在沿海以及一些两个内陆省份（四川，湖南）

In [22]:
data['订单创建时间'] = pd.to_datetime(data['订单创建时间'])
data['订单付款时间'] = pd.to_datetime(data['订单付款时间'])
result31 = data.groupby(data['订单创建时间'].apply(lambda x: x.strftime("%Y-%m-%d"))).agg({'订单编号':'count'}).to_dict()['订单编号']
c = (
    Line()
    .add_xaxis(list(result31.keys()))
    .add_yaxis("订单量", list(result31.values()))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
            ]
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="每日订单量走势"))
)
c.render_notebook()

从上图来看，2月份上半月由于受新冠疫情影响，订单量比较少，随着复工开展，下半月的订单量增长明显。预计在之后随着疫情减轻，订单量还会上升。

In [24]:
result32 = data.groupby(data['订单创建时间'].apply(lambda x: x.strftime("%H"))).agg({'订单编号':'count'}).to_dict()['订单编号']
x = [*result32.keys()]
y = [*result32.values()]
c = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("订单量", y)
    .set_global_opts(title_opts=opts.TitleOpts(title="每小时订单量走势"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="峰值"),
                opts.MarkPointItem(name="第二峰值", coord=[x[15], y[15]], value=y[15]),
                opts.MarkPointItem(name="第三峰值", coord=[x[10], y[10]], value=y[10]),
            ]
        ),
    )
)
c.render_notebook()

从每小时订单量走势来看，一天中有3个高峰期（10点、15点、21点），其中21点-22点之间是一天中订单量最多的时候。对于卖家的指导意义就是，为了提高订单量，高峰期时应该尽量保证客服的回复速度，尤其是晚上21点-22点之间，所以很多做电商的基本都有夜班。同时对于网站来说，这几个峰值时间的流量也也应在网站的最大承受范围内，要据此维护网站。

In [25]:
s = data['订单付款时间'] - data['订单创建时间']
s[s.notnull()].apply(lambda x: x.seconds / 60 ).mean()  # 从下单到付款的平均耗时为 7.7 分钟

7.73990465119536

在付款完成的订单内，从下单到付款的平均耗时为7.7分钟